In [ ]:
# 导入包(PypI)
import os
import re
import time
import json
import random
import pymongo
import pandas as pd

from PIL import Image
from io import BytesIO
from datetime import datetime
from tqdm.notebook import tqdm

from scrapy.selector import Selector
from scrapy.http import TextResponse

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import requests

In [ ]:
def scroll_until_posts_loaded(browser, target_post_count, max_scroll_attempts=30):
    """Scroll until the target number of posts is loaded or the max attempts are reached."""
    posts_seen = 0
    scroll_attempts = 0

    while posts_seen < target_post_count and scroll_attempts < max_scroll_attempts:
        # Scroll to the bottom of the page
        browser.execute_script("window.scrollBy(0, 1000);")
        time.sleep(2)  # Give time for new posts to load

        # Count the number of posts currently loaded
        posts = browser.find_elements(By.XPATH, '//div[contains(@data-pagelet, "FeedUnit")]')
        new_posts_seen = len(posts)

        if new_posts_seen > posts_seen:
            posts_seen = new_posts_seen
            print(f"Posts loaded: {posts_seen}")
        else:
            # If no new posts load, break the loop
            print("No new posts loaded. Stopping scroll.")
            break

        scroll_attempts += 1

    print(f"Final post count: {posts_seen}")
    return posts_seen

def download_image(url, save_path):
        """
        下载图片并保存到指定路径。

        Args:
            url (str): 图片的URL地址。
            save_path (str): 图片保存的本地路径。

        Returns:
            bool: 下载是否成功。
        """
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # 检查请求是否成功
            with open(save_path, 'wb') as f:
                f.write(response.content)
            return True
        except Exception as e:
            print(f"下载图片时出错，URL: {url}, 错误: {str(e)}")
            return False

def scrape_instagram(link = "https://www.instagram.com/nyushanghai/", num_posts_to_scrape = 10):
    browser.get(link)
    scroll_until_posts_loaded(browser, 1, 30)

    # Initialize storage lists
    post_number = [i for i in range(num_posts_to_scrape)]
    content_list = []
    reactions_list = []
    comments_list = []
    media_type_list = []
    image_url_list = []
    video_url_list = []

    wait_time = random.uniform(3, 5)  # Random wait time for scrolling

    num_posts_to_scrape = num_posts_to_scrape

    # Loop to scroll and capture posts
    collected_posts_by_3 = 1

    while collected_posts_by_3 < num_posts_to_scrape/3:
        browser.execute_script("window.scrollBy(0, 500);")
        time.sleep(wait_time)

        response = TextResponse(url=browser.current_url, body=browser.page_source.encode('utf-8'), encoding='utf-8')
        selector = Selector(response)
        collected_posts_by_3 += 1

    post_class = "x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz _a6hd"
    post_links = selector.xpath(
            f'//a[contains(@class, "{post_class}")]/@href'
        ).getall()[1:num_posts_to_scrape+1]
    post_links = [f'https://www.instagram.com{link}' for link in post_links]
    print(post_links)

    # loop through each post and retrieve likes and image urls.
    for link in post_links[:num_posts_to_scrape]:
        # Execute JavaScript to open a new tab
        base_url = link

        # List to store image sources
        filtered_image_srcs = []

        image_idx = 1

        browser.get(link)
        time.sleep(5)

        # Parse the page source
        response = TextResponse(
            url=browser.current_url,
            body=browser.page_source.encode('utf-8'),
            encoding='utf-8'
        )
        selector = Selector(response)

        # Extract h1 with the specific class
        h1_text_parts = selector.xpath('//h1[contains(@class, "_ap3a") and contains(@class, "_aaco") and contains(@class, "_aacu") and contains(@class, "_aacx") and contains(@class, "_aad7") and contains(@class, "_aade")]//text()').getall()
        # Join all parts into one string, stripping extra spaces
        h1_text = ' '.join([text.strip() for text in h1_text_parts if text.strip()])
        content_list.append(h1_text)

        # Extract likes
        span_text = selector.xpath('//span[contains(@class, "html-span") and contains(@class, "xdj266r") and contains(@class, "x11i5rnm") and contains(@class, "xat24cr") and contains(@class, "x1mh8g0r") and contains(@class, "xexx8yu") and contains(@class, "x4uap5") and contains(@class, "x18d9i69") and contains(@class, "xkhd6sd") and contains(@class, "x1hl2dhg") and contains(@class, "x16tdsg8") and contains(@class, "x1vvkbs")]/text()').get()
        reactions_list.append(span_text)

        # find vedio urls
        if "/reel/" in link:
            media_type_list.append("vedio")
            image_url_list.append(None)
            continue

        if browser.current_url.endswith("?img_index=1"):
            # Iterate over img_index values
            for img_index in range(1, 21, 2):  # Check up to 10 images (adjust range as needed)
                img_url = f"{base_url}?img_index={img_index}"
                print(f"Accessing: {img_url}")

                # Open the URL in the browser
                browser.get(img_url)
                time.sleep(5)  # Wait for the page to load

                # early stop for the image search
                if browser.current_url.endswith("?img_index=1") and img_index != 1:
                    print("Detected reset to img_index=1. No more images found.")
                    break

                # Parse the current page source
                response = TextResponse(
                    url=browser.current_url,
                    body=browser.page_source.encode('utf-8'),
                    encoding='utf-8'
                )
                selector = Selector(response)

                # Extract image sources from the current index
                image_srcs = selector.xpath(
                    './/img[@src][not(ancestor::div[contains(@class, "x1qjc9v5 x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1lliihq xdt5ytf x2lah0s xln7xf2 xk390pu xdj266r x11i5rnm xat24cr x1mh8g0r x4uap5 x18d9i69 xkhd6sd x24vp2c x1n2onr6 x11njtxf")])]/@src'
                ).getall()
                valid_images = [src for src in image_srcs if 'data' not in src and '2885-19' not in src]

                filtered_image_srcs.extend(valid_images)
                print(filtered_image_srcs)


            media_type_list.append("img")
            image_url_list.append(filtered_image_srcs)

        else:
            # Parse the current page source
            response = TextResponse(
                url=browser.current_url,
                body=browser.page_source.encode('utf-8'),
                encoding='utf-8'
            )
            selector = Selector(response)

            # Extract image sources from the current index
            image_srcs = selector.xpath(
                './/img[@src][not(ancestor::div[contains(@class, "x1qjc9v5 x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1lliihq xdt5ytf x2lah0s xln7xf2 xk390pu xdj266r x11i5rnm xat24cr x1mh8g0r x4uap5 x18d9i69 xkhd6sd x24vp2c x1n2onr6 x11njtxf")])]/@src'
            ).getall()
            valid_images = [src for src in image_srcs if 'data' not in src and '2885-19' not in src]


            filtered_image_srcs.extend(valid_images)

            media_type_list.append("img")
            image_url_list.append(filtered_image_srcs)

    browser.get('https://www.instagram.com/nyushanghai/')

    # 下载主图片
    print("开始下载帖子图片")
    image_download_bar = tqdm(total=len(image_url_list), desc="下载主图片")
    for idx_post, image_url_post in enumerate(image_url_list):
        if image_url_post == None:
            image_download_bar.update(1)
            continue
        folder_name = f"data/{idx_post}"
        os.makedirs(folder_name, exist_ok=True)
        for idx_image, url in enumerate(list(set(image_url_post))):
            # 构造图片保存路径，使用note_id作为文件名
            image_extension = '.png'   # 获取文件扩展名
            # if image_extension.lower() not in ['.jpg', '.jpeg', '.png', '.gif']:
            #     image_extension = '.jpg'  # 默认使用.jpg
            image_filename = f"{folder_name}/image_{idx_image}{image_extension}"
            success = download_image(url, image_filename)
            if not success:
                image_filename = "N/A"  # 如果下载失败，标记为N/A
        image_download_bar.update(1)
    image_download_bar.close()


    # Save Data to CSV
    df = pd.DataFrame({
        'Row Number': post_number,
        'Content': content_list,
        'Likes': reactions_list,
        'Images': image_url_list,
        'Post Type': media_type_list
    })
    df = pd.DataFrame(df)
    print(df.head())
    df.to_csv('scraped_data.csv', encoding='utf-8-sig', index = False)
    print("数据已保存到 'scraped_data.csv'")

    return df

In [ ]:
# 进入facebook网页，登录账号
# 配置Chrome浏览器
service = Service('/Users/apple/.wdm/drivers/chromedriver/mac64/128.0.6613.138/chromedriver-mac-x64/chromedriver')
browser = webdriver.Chrome(service=service)


browser.get('https://www.instagram.com/nyushanghai/')

wait = WebDriverWait(browser, 60)  # 设置最长等待时间为60秒

In [ ]:
scrape_instagram()

No new posts loaded. Stopping scroll.
Final post count: 0
['https://www.instagram.com/nyushanghai/p/DEWTg5AhNCK/', 'https://www.instagram.com/nyush_robotics_club/p/DEUj5b_P2vG/', 'https://www.instagram.com/imalowres.nyu/reel/DEQwOL6x_OA/', 'https://www.instagram.com/nyushanghai/p/DEP6ZNaiOQ0/', 'https://www.instagram.com/nyushanghai/reel/DEPfGRqqIvB/', 'https://www.instagram.com/nyushanghai/reel/DEMWU8gNT-t/', 'https://www.instagram.com/tedxnyushanghai/reel/DEKJDKNSE6i/', 'https://www.instagram.com/nyushanghai/p/DEBN1RHCimi/', 'https://www.instagram.com/nyushanghai/reel/DD-iLUoBbWC/', 'https://www.instagram.com/nyushanghai/reel/DD8E5THJOPf/']
Accessing: https://www.instagram.com/nyushanghai/p/DEWTg5AhNCK/?img_index=1
['https://scontent-lga3-1.cdninstagram.com/v/t51.29350-15/472299615_1534455797215143_6544459260303951891_n.jpg?stp=dst-jpg_e35_tt6&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi4xNDQweDEwODMuc2RyLmYyOTM1MC5kZWZhdWx0X2ltYWdlIn0&_nc_ht=scontent-lga3-1.cdninstagram.com&_nc_cat=10

下载主图片:   0%|          | 0/10 [00:00<?, ?it/s]

   Row Number                                            Content Likes  \
0           0  🩰+🤖=💡 What happens when motion capture, machin...   250   
1           1  NYUSH Robotics team visited Wuxi this week to ...   314   
2           2  🌏✨ Here’s to 2025! 🎉 From late-night project b...   112   
3           3  🥂 Cheers to a bright and inspiring 2025! We wi...   233   
4           4  And just like that, NYU Shanghai’s 2024 is com...   202   

                                              Images Post Type  
0  [https://scontent-lga3-1.cdninstagram.com/v/t5...       img  
1  [https://scontent-lga3-1.cdninstagram.com/v/t5...       img  
2                                               None     vedio  
3  [https://scontent-lga3-3.cdninstagram.com/v/t3...       img  
4                                               None     vedio  
数据已保存到 'scraped_data.csv'


,Row Number,Content,Likes,Images,Post Type
0,0,"🩰+🤖=💡 What happens when motion capture, machin...",250,[https://scontent-lga3-1.cdninstagram.com/v/t5...,img
1,1,NYUSH Robotics team visited Wuxi this week to ...,314,[https://scontent-lga3-1.cdninstagram.com/v/t5...,img
2,2,🌏✨ Here’s to 2025! 🎉 From late-night project b...,112,None,vedio
3,3,🥂 Cheers to a bright and inspiring 2025! We wi...,233,[https://scontent-lga3-3.cdninstagram.com/v/t3...,img
4,4,"And just like that, NYU Shanghai’s 2024 is com...",202,None,vedio
5,5,POV: You're hanging out with Elima Zamirbekova...,342,None,vedio
6,6,✨ “What happens when we let serendipity lead t...,48,None,vedio
7,7,One of the coolest things about being a studen...,441,[https://scontent-lga3-1.cdninstagram.com/v/t3...,img
8,8,"Happy Holidays, everyone! 🎄 Cheers to creating...",154,None,vedio
9,9,Have you been to NYU Shanghai’s community roof...,132,None,vedio
